## Track latency of single GOES product.

When executed, the code will wait till the next GOES image has been acquired, and is made available in AWS bucket.

NOTE: All time stamps in UTC.

In [1]:
import os
import time
import datetime

import pandas

import helper

In [2]:
check_ever_n_seconds = 1

DOWNLOAD_DIR = "../DataDump//"

radProduct = 'ABI-L1b-RadC' ## L1 products CONUS

## Level 2 data product to check
dataProductCode = [ 'ABI-L2-MCMIPC', ## Level 2 Cloud and Moisture Imagery CONUS
                    'ABI-L2-ACMC',   ## Level 2 Clear Sky Mask CONUS
                    'ABI-L2-FDCC'    ## Level 2 Fire (Hot Spot Characterization) CONUS
                ]

In [3]:
## var init
counter = 0
dict_product_avaliable_time = {}

I suggest to run the sections below in quick sucession. Or, run all cells at once.

In [4]:
queryTime = datetime.datetime.now(datetime.timezone.utc)


## Get a Time Range to look for past/old imagery already available --> 
## Here we look back 30 minutes
past_time, _ = helper.getTimeRange(queryTime, bufferMinutes = [30, 0])



## Also, we would next look forward in time () till we get an image
## This step is a hack to extend functions for historic images to query future image
## start_time --> would be current time, or query time
## end_time   --> is a time 10 minutes in future
start_time, end_time = helper.getTimeRange(queryTime, bufferMinutes = [0, 10])


print("UTC Start Time  -->", start_time)
print("UTC End Time    -->", end_time)



UTC Start Time  --> 2022-08-01 14:21:31.569498+00:00
UTC End Time    --> 2022-08-01 14:31:31.569498+00:00


In [5]:
## Here we are not downloading the old images -- just querying the available imagery in AWS.
for p in dataProductCode:
    df_old = helper.getProductList(past_time, start_time, product = p, satellite = 16)

    print("{}:".format(p))
    print(df_old['start'].unique())
    print('-------------------')

    df_old.to_csv("{}_old_data_availiblity.csv".format(p))

ABI-L2-MCMIPC:
<DatetimeArray>
['2022-08-01 13:56:17.300000+00:00', '2022-08-01 14:01:17.300000+00:00',
 '2022-08-01 14:06:17.300000+00:00', '2022-08-01 14:11:17.300000+00:00',
 '2022-08-01 14:16:17.300000+00:00']
Length: 5, dtype: datetime64[ns, UTC]
-------------------
ABI-L2-ACMC:
<DatetimeArray>
['2022-08-01 13:56:17.300000+00:00', '2022-08-01 14:01:17.300000+00:00',
 '2022-08-01 14:06:17.300000+00:00', '2022-08-01 14:11:17.300000+00:00',
 '2022-08-01 14:16:17.300000+00:00']
Length: 5, dtype: datetime64[ns, UTC]
-------------------
ABI-L2-FDCC:
<DatetimeArray>
['2022-08-01 13:56:17.300000+00:00', '2022-08-01 14:01:17.300000+00:00',
 '2022-08-01 14:06:17.300000+00:00', '2022-08-01 14:11:17.300000+00:00',
 '2022-08-01 14:16:17.300000+00:00']
Length: 5, dtype: datetime64[ns, UTC]
-------------------


### Check for Level 1 products that have been accquired running the notebook
--> If the product is not available, wait till all 16 bands are available


In [6]:
## Keep pinging AWS to check for a new product every check_ever_n_seconds
## We are looking for Level 1 product

while True:

    df = helper.getProductList(start_time, end_time, product = radProduct, satellite = 16)

    # Once all 16 bands are available
    if len(df) >= 16:

        # Note the time 
        dict_product_avaliable_time[radProduct] = datetime.datetime.now(datetime.timezone.utc)

        print('')
        print("Frame Start: ", df['start'].mean().strftime('%H:%M:%S.%f'), '+/-', df['start'].std().seconds + df['start'].std().microseconds/1E6)
        print("Frame End  : ", df['end'].mean().strftime('%H:%M:%S.%f'),   '+/-', df['end'].std().seconds + df['end'].std().microseconds/1E6  )
        print('')
        
        frameEndTime = df['end'].mean()
        print("Level 1 Created           --> {} --> {} seconds.".format(df.iloc[-1]['creation'], (df.iloc[-1]['creation'] - frameEndTime).seconds))
        print("Level 1 Available for us  --> {} --> {} seconds.".format(dict_product_avaliable_time[radProduct], (dict_product_avaliable_time[radProduct] - frameEndTime).seconds))
        counter += 1

        break
    
    
    time.sleep(check_ever_n_seconds)



Frame Start:  14:26:17.300000 +/- 0.0
Frame End  :  14:28:55.049999 +/- 0.513809

Level 1 Created           --> 2022-08-01 14:29:02+00:00 --> 6 seconds.
Level 1 Available for us  --> 2022-08-01 14:29:19.142033+00:00 --> 24 seconds.


### Check for relevant L2 
--> If the product is not available, wait

In [7]:
while counter < len(dataProductCode):

    ## Reset counter
    counter = 0

    for p in dataProductCode:
        
        
        df = helper.getProductList(start_time, end_time, product = p, satellite = 16)

        if len(df) > 0:

            if p not in dict_product_avaliable_time.keys():
                dict_product_avaliable_time[p] = datetime.datetime.now(datetime.timezone.utc)
            
                print(df.iloc[-1]['file'])
                print("Level 2 {} Created          --> {} --> {} seconds.".format(p, df.iloc[-1]['creation'], (df.iloc[-1]['creation'] - frameEndTime).seconds))
                print("Level 2 {} Available for us --> {} --> {} seconds.".format(p, dict_product_avaliable_time[p], (dict_product_avaliable_time[p] - frameEndTime).seconds))
                print("-------------------------------\n")
            counter += 1  
    
    time.sleep(check_ever_n_seconds)


noaa-goes16/ABI-L2-FDCC/2022/213/14/OR_ABI-L2-FDCC-M6_G16_s20222131426173_e20222131428546_c20222131429170.nc
Level 2 ABI-L2-FDCC Created          --> 2022-08-01 14:29:17+00:00 --> 21 seconds.
Level 2 ABI-L2-FDCC Available for us --> 2022-08-01 14:29:23.589821+00:00 --> 28 seconds.
-------------------------------

noaa-goes16/ABI-L2-MCMIPC/2022/213/14/OR_ABI-L2-MCMIPC-M6_G16_s20222131426173_e20222131428558_c20222131429056.nc
Level 2 ABI-L2-MCMIPC Created          --> 2022-08-01 14:29:05.600000+00:00 --> 10 seconds.
Level 2 ABI-L2-MCMIPC Available for us --> 2022-08-01 14:29:24.869660+00:00 --> 29 seconds.
-------------------------------

noaa-goes16/ABI-L2-ACMC/2022/213/14/OR_ABI-L2-ACMC-M6_G16_s20222131426173_e20222131428546_c20222131430083.nc
Level 2 ABI-L2-ACMC Created          --> 2022-08-01 14:30:08.300000+00:00 --> 73 seconds.
Level 2 ABI-L2-ACMC Available for us --> 2022-08-01 14:30:25.835199+00:00 --> 90 seconds.
-------------------------------



## Download Level 1 RAD CONUS data


In [8]:
df = helper.getProductList(start_time, end_time, product = 'ABI-L1b-RadC', satellite = 16)
df = df[df["band"].isin([2, 7, 13, 14, 15])]

uniqueStartTimes = df['start'].unique()
print(f'{len(uniqueStartTimes)} unique image acquisitions found.')

helper.download(df = df, BASEDIR = DOWNLOAD_DIR, startTime = uniqueStartTimes[-1])

print("L1 RAD Download Done on: {}".format(datetime.datetime.now(datetime.timezone.utc)))

1 unique image acquisitions found.
L1 RAD Download Done on: 2022-08-01 14:30:39.886833+00:00


## Download Level 2 Cloud and Moisture Imagery CONUS data

In [9]:
df = helper.getProductList(start_time, end_time, product = 'ABI-L2-MCMIPC', satellite = 16)

uniqueStartTimes = df['start'].unique()
print(f'{len(uniqueStartTimes)} unique image acquisitions found.')

helper.download(df = df, BASEDIR = DOWNLOAD_DIR, startTime = uniqueStartTimes[-1])

print("L2 MCMIPC Download Done on: {}".format(datetime.datetime.now(datetime.timezone.utc)))

1 unique image acquisitions found.
L2 MCMIPC Download Done on: 2022-08-01 14:30:49.998493+00:00
